In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chuny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chuny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!pip install networkx gensim

In [3]:
import networkx as nx
import numpy as np
import os
import time
import pandas as pd
from itertools import chain
from collections import defaultdict
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

# Pipeline

### Data Loader

We need to load networks into memory. Usually networks are organized as pairs of nodes. And sometimes different edges have different weights. Hence, we use networkx.DiGraph to store such structure information and attributes.

In [4]:
import random

def load_data(file_name):
    """
    read edges from an edge file
    """
    edges = list()
    df = pd.read_csv(file_name)
    for idx, row in df.iterrows():
        user_id, friends = row["user_id"], eval(row["friends"])
        for friend in friends:
            # add each friend relation as an edge
            ########## begin ##########
            edges.append((user_id, friend))
            ########## end ##########
    edges = sorted(edges)

    return edges

def generate_false_edges(true_edges, num_false_edges=5):
    """
    generate false edges given true edges
    """
    nodes = list(set(chain.from_iterable(true_edges)))
    true_edges = set(true_edges)
    false_edges = set()
    while len(false_edges) < num_false_edges:
        # randomly sample two different nodes and check whether the pair exisit or not
        # check 3 things:
        #   if head == tail, then it is not an edge.
        #   if the edge is in true_edges, skip.
        #   if the edge is in false edges, skip.
        ########## begin ##########
        head = random.choice(nodes)
        tail = random.choice(nodes)
        if head != tail and (head, tail) not in true_edges and (head, tail) not in false_edges:
            false_edges.add((head, tail))
        ########## end ##########

    false_edges = sorted(false_edges)
    return false_edges

def construct_graph_from_edges(edges):
    """
    generate a directed graph object given true edges
    DiGraph documentation: https://networkx.github.io/documentation/stable/reference/classes/digraph.html
    """
    # convert a list of edges {(u, v)} to a list of edges with weights {(u, v, w)}
    edge_weight = defaultdict(float)

    ########## begin ##########
    # 1. For each edge, add 1.0 to the edge_weight

    for edge in edges:
        edge_weight[edge] += 1.0

    ###########################
    # 2. In a sorted order of edges, make a list of weighted_edge_list and weight_list.
    #   - weighted_edge_list: list[tuple], with each element having a format (node_1, node_2, weight)
    #   - weight_list: list[int]
    weighted_edge_list = list()
    weight_list = []

    for edge in sorted(edge_weight.keys()):
        weight = edge_weight[edge]
        weighted_edge_list.append((edge[0], edge[1], weight))
        weight_list.append(weight)

    ########## end ##########

    weights = np.asarray(weight_list)
    print(f"edges have mean weights {weights.mean()} with std {weights.std()}")
    graph = nx.DiGraph()
    graph.add_weighted_edges_from(weighted_edge_list)

    print("number of nodes:", graph.number_of_nodes())
    print("number of edges:", graph.number_of_edges())

    return graph

### Random Walk Generator

Random walk generators or random walkers yield random walks that contain both local and higher-order neighborhood information. However, naive non-uniform sampling is very slow, which requires O(n) time complexity. Here alias sampling can reduce the time complexity to O(1) with O(n) space. If you are interested, please see the following blog.

In [5]:
def alias_setup(probs):
    """
    compute utility lists for non-uniform sampling from discrete distributions.
    details: https://lips.cs.princeton.edu/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    """
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=int)

    smaller = list()
    larger = list()
    for kk, prob in enumerate(probs):
        q[kk] = K * prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def get_alias_node(graph, node):
    """
    get the alias node setup lists for a given node.
    """
    # get the unnormalized probabilities with the first-order information
    unnormalized_probs = list()

    ########## begin ##########
    for nbr in graph.neighbors(node):
        unnormalized_probs.append(graph[node][nbr]['weight'])
    ########## end ##########

    unnormalized_probs = np.array(unnormalized_probs)
    if len(unnormalized_probs) > 0:
        normalized_probs = unnormalized_probs / unnormalized_probs.sum()
    else:
        normalized_probs = unnormalized_probs

    return alias_setup(normalized_probs)

def get_alias_edge(graph, src, dst, p=1, q=1):
    """
    get the alias edge setup lists for a given edge.
    """
    # get the unnormalized probabilities with the second-order information
    unnormalized_probs = list()

    ########## begin ##########
    for dst_nbr in graph.neighbors(dst):
        if dst_nbr == src: # distance is 0
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'] / p)
        elif graph.has_edge(dst_nbr, src): # distance is 1
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'])
        else: # distance is 2
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'] / q)
    ########## end ##########


    unnormalized_probs = np.array(unnormalized_probs)
    if len(unnormalized_probs) > 0:
        normalized_probs = unnormalized_probs / unnormalized_probs.sum()
    else:
        normalized_probs = unnormalized_probs

    return alias_setup(normalized_probs)

def preprocess_transition_probs(graph, p=1, q=1):
    """
    preprocess transition probabilities for guiding the random walks.
    """
    alias_nodes = dict()
    for node in graph.nodes():
        alias_nodes[node] = get_alias_node(graph, node)

    alias_edges = dict()
    for edge in graph.edges():
        alias_edges[edge] = get_alias_edge(graph, edge[0], edge[1], p=p, q=q)

    return alias_nodes, alias_edges

The difference between DeepWalk and node2vec is how to generate random walks. The former only consider the first-order information while the latter also involves the second-order information.

In [6]:
def alias_draw(J, q):
    """
    draw sample from a non-uniform discrete distribution using alias sampling.
    """
    K = len(J)

    kk = int(np.floor(np.random.rand() * K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]


# helper function to generate the long random walk as desired
def fallback(walk, fetch_last_num=1):
    if len(walk) > fetch_last_num:
        walk.pop()
        fetched = []
        for i in range(fetch_last_num):
            fetched.append(walk[-1-i])
        return walk, fetched
    else:
        return [], [None for _ in range(fetch_last_num)]

def generate_first_order_random_walk(graph, alias_nodes,
                                     walk_length=10, start_node=None, verbose=False, max_trails=10):
    """
    simulate a random walk starting from start node and considering the first order information.
    max_trials: set the max trials to be one for standard random walk. Larger max_trails will make the generated biased.
    """
    if start_node == None:
        start_node = np.random.choice(graph.nodes())
    walk = [start_node]
    cur = start_node
    num_tried = 0

    ########## begin ##########
    while len(walk) < walk_length:
        cur_nbrs = list(graph.neighbors(cur))
        if len(cur_nbrs) > 0: # if we can sample next nodes
            # sample the next node based on alias_nodes
            cur = cur_nbrs[alias_draw(*alias_nodes[cur])]
            walk.append(cur)
        else: # if we can't do that
            num_tried += 1
            if num_tried >= max_trails:
                break

            walk, fetched = fallback(walk, fetch_last_num=1)
            cur = fetched[0]
            if len(walk) == 0: # if falls back to the empty walk
                start_node = np.random.choice(graph.nodes())
                walk = [start_node]
                cur = start_node
    ########## end ##########

    if verbose:
        print(f'walk of lenght {len(walk)} generated with {num_tried} trails')
    return walk

def generate_second_order_random_walk(graph, alias_nodes, alias_edges,
                                      walk_length=10, start_node=None, verbose=False, max_trails=10):
    """
    simulate a random walk starting from start node and considering the second order information.
    """
    if start_node == None:
        start_node = np.random.choice(graph.nodes())
    walk = [start_node]

    prev = None
    cur = start_node
    num_tried = 0

    ########## begin ##########
    while len(walk) < walk_length:
        cur_nbrs = list(graph.neighbors(cur))
        if len(cur_nbrs) > 0:
            if prev is None:
                # sample the next node based on alias_nodes
                cur = cur_nbrs[alias_draw(*alias_nodes[cur])]
            else:
                # sample the next node based on alias_edges
                cur = cur_nbrs[alias_draw(*alias_edges[(prev, cur)])]
            walk.append(cur)
            prev = walk[-2]
        else:
            num_tried += 1
            if num_tried >= max_trails:
                break
            walk, (cur, prev) = fallback(walk, fetch_last_num=2)
            if len(walk) == 0:
                start_node = np.random.choice(graph.nodes())
                walk = [start_node]
                cur = start_node
                prev = None
    ########## end ##########
    if verbose:
        print(f'walk of lenght {len(walk)} generated with {num_tried} trails')
    return walk

### Network Embedding Algorithms

In [7]:
def build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=10, walk_length=10, negative=10):
    """
    build a deepwalk model
    """
    print("building a DeepWalk model...", end="\t")
    st = time.time()
    np.random.seed(0)
    nodes = list(graph.nodes())
    walks = list()
    # generate random walks
    for walk_iter in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(generate_first_order_random_walk(graph, alias_nodes,
                                                          walk_length=walk_length, start_node=node))

    walk_lens = [len(w) for w in walks]
    if len(walk_lens) > 0:
        avg_walk_len = sum(walk_lens) / len(walk_lens)
    else:
        avg_walk_len = 0.0
    print("number of walks: %d\taverage walk length: %.4f" % (len(walks), avg_walk_len), end="\t")

    # train a skip-gram model for these walks
    model = Word2Vec(walks, vector_size=node_dim, window=3, min_count=0, sg=1, negative=negative, workers=os.cpu_count(), epochs=10)
    print("build time: %.4f" % (time.time()-st))

    return model

def build_node2vec(graph, alias_nodes, alias_edges, node_dim=10, num_walks=10, walk_length=10, negative=10):
    """
    build a node2vec model
    """
    print("building a node2vec model...", end="\t")
    st = time.time()
    np.random.seed(0)
    nodes = list(graph.nodes())
    walks = list()
    # generate random walks
    for walk_iter in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(generate_second_order_random_walk(graph, alias_nodes, alias_edges,
                                                           walk_length=walk_length, start_node=node))

    walk_lens = [len(w) for w in walks]
    if len(walk_lens) > 0:
        avg_walk_len = sum(walk_lens) / len(walk_lens)
    else:
        avg_walk_len = 0.0
    print("number of walks: %d\taverage walk length: %.4f" % (len(walks), avg_walk_len), end="\t")

    # train a skip-gram model for these walks
    model = Word2Vec(walks, vector_size=node_dim, window=3, min_count=0, sg=1, negative=negative, workers=os.cpu_count(), epochs=10, seed=1)
    print("build time: %.4f" % (time.time()-st))

    return model

### Scorer

In [8]:
def get_cosine_sim(model, u, v):
    """
    get the cosine similarity between two nodes
    """
    try:
        u = model.wv[u]
        v = model.wv[v]
        return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    except:
        return 0

def get_auc_score(model, true_edges, false_edges):
    """
    get the auc score
    """
    y_true = [1] * len(true_edges) + [0] * len(false_edges)

    y_score = list()
    for e in true_edges:
        y_score.append(get_cosine_sim(model, e[0], e[1]))
    for e in false_edges:
        y_score.append(get_cosine_sim(model, e[0], e[1]))

    return roc_auc_score(y_true, y_score)

### Try them over a Real-life Network

Firstly, we need to load edges into memory and use the networkx.DiGraph structure to store the graph.

In [9]:
user_file = "data/user.csv"
edges = load_data(user_file)
graph = construct_graph_from_edges(edges)

edges have mean weights 1.0 with std 0.0
number of nodes: 3930
number of edges: 15688


After that, we can use preprocess transition probabilities with the help of alias sampling.

In [10]:
alias_nodes, alias_edges = preprocess_transition_probs(graph, p=2, q=2)

We can use random walk generators to generate random walks.

Let's try to generate a first-order random walk and a second-order random walk.

In [11]:
generate_first_order_random_walk(graph, alias_nodes=alias_nodes,
                                 start_node="N6ZTMIue-2b30CJv2tyPGg", walk_length=10)

['N6ZTMIue-2b30CJv2tyPGg',
 'gqL5KBs2oS7qobnyd99iKg',
 'zTK1nPD2Hpa-ksSXsE-JzQ',
 'gqL5KBs2oS7qobnyd99iKg',
 'AmMd7xpnaf8axS_roCBFRw',
 'YBT3EKUNN4IP8m4x7sGu1g',
 'AmMd7xpnaf8axS_roCBFRw',
 'G-6X-llgA_qAxGxocykHzQ',
 'AmMd7xpnaf8axS_roCBFRw',
 'wUgRsMwL-BCreuMBgmFdWg']

In [12]:
generate_second_order_random_walk(graph, alias_nodes=alias_nodes, alias_edges=alias_edges,
                                  start_node="N6ZTMIue-2b30CJv2tyPGg", walk_length=10)

['N6ZTMIue-2b30CJv2tyPGg',
 'gqL5KBs2oS7qobnyd99iKg',
 'juLlBJ6SQMyMLxPLczZxNA',
 'gqL5KBs2oS7qobnyd99iKg',
 'zTK1nPD2Hpa-ksSXsE-JzQ',
 'MQwSyZ2MZ6N7rtAmphZCow',
 'ZZoxuBqon2tTMST4YBWa8w',
 'ZljxRipwkMk9u3JXEi67iQ',
 'el3TmKFEFzZOcNbCw2FNlQ',
 '3NnPbhmv_vEfPTBp2pnn9Q']

And we can build a DeepWalk model and a node2vec model. Here we set p=q=0.5 so that the walker will not go very far away from the start node.

In [13]:
deepwalk = build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=10, walk_length=10)

building a DeepWalk model...	number of walks: 39300	average walk length: 8.7485	build time: 9.8894


In [14]:
node2vec = build_node2vec(graph, alias_nodes, alias_edges, node_dim=10, num_walks=10, walk_length=10)

building a node2vec model...	number of walks: 39300	average walk length: 9.8997	build time: 18.9038


Let's see the node embeddings of three nodes, and cosine similarities of two edges.

In [15]:
print('--- Deepwalk (1st order random walk) ---')
print("node embedding (\"N6ZTMIue-2b30CJv2tyPGg\"):",
      deepwalk.wv["N6ZTMIue-2b30CJv2tyPGg"])
print("node embedding (\"N7E-CfqdME28dakWdEKNvw\"):",
      deepwalk.wv["N7E-CfqdME28dakWdEKNvw"])
print("node embedding (\"MmlJSLDg-IFaeXb5wdJbgg\"):",
      deepwalk.wv["MmlJSLDg-IFaeXb5wdJbgg"])
print("true edge (\"N6ZTMIue-2b30CJv2tyPGg\", \"N7E-CfqdME28dakWdEKNvw\"):",
      get_cosine_sim(deepwalk, "N6ZTMIue-2b30CJv2tyPGg", "N7E-CfqdME28dakWdEKNvw"))
print("false edge (\"N6ZTMIue-2b30CJv2tyPGg\", \"MmlJSLDg-IFaeXb5wdJbgg\"):",
      get_cosine_sim(deepwalk, "N6ZTMIue-2b30CJv2tyPGg", "MmlJSLDg-IFaeXb5wdJbgg"))

print()
print('--- Node2Vec (2nd order random walk) ---')
print("node embedding (\"N6ZTMIue-2b30CJv2tyPGg\"):",
      node2vec.wv["N6ZTMIue-2b30CJv2tyPGg"])
print("node embedding (\"N7E-CfqdME28dakWdEKNvw\"):",
      node2vec.wv["N7E-CfqdME28dakWdEKNvw"])
print("node embedding (\"MmlJSLDg-IFaeXb5wdJbgg\"):",
      node2vec.wv["MmlJSLDg-IFaeXb5wdJbgg"])
print("true edge (\"N6ZTMIue-2b30CJv2tyPGg\", \"N7E-CfqdME28dakWdEKNvw\"):",
      get_cosine_sim(node2vec, "N6ZTMIue-2b30CJv2tyPGg", "N7E-CfqdME28dakWdEKNvw"))
print("false edge (\"N6ZTMIue-2b30CJv2tyPGg\", \"MmlJSLDg-IFaeXb5wdJbgg\"):",
      get_cosine_sim(node2vec, "N6ZTMIue-2b30CJv2tyPGg", "MmlJSLDg-IFaeXb5wdJbgg"))

--- Deepwalk (1st order random walk) ---
node embedding ("N6ZTMIue-2b30CJv2tyPGg"): [-0.10837889 -0.10652022  1.4146445   0.8012465   0.65344757 -0.14094147
  0.7993037   0.68759024 -0.30896777 -0.82715493]
node embedding ("N7E-CfqdME28dakWdEKNvw"): [-0.4287536   0.22108056  1.907664    0.22008018  1.1516143   0.24478206
  1.3893412   1.2056402  -1.047422   -0.16928416]
node embedding ("MmlJSLDg-IFaeXb5wdJbgg"): [ 0.6157956  -1.3259395   1.6965528   1.166923   -0.84236723  2.593628
  1.9057      0.58812636 -2.002583    0.7257554 ]
true edge ("N6ZTMIue-2b30CJv2tyPGg", "N7E-CfqdME28dakWdEKNvw"): 0.85878074
false edge ("N6ZTMIue-2b30CJv2tyPGg", "MmlJSLDg-IFaeXb5wdJbgg"): 0.4199828

--- Node2Vec (2nd order random walk) ---
node embedding ("N6ZTMIue-2b30CJv2tyPGg"): [-0.21442583 -0.8520855   1.712711   -0.20094264 -0.89525235  0.9691995
  0.93174154  0.43016255 -0.11394635 -0.02940412]
node embedding ("N7E-CfqdME28dakWdEKNvw"): [ 0.18206334 -1.7999327   1.3429816   0.14228307 -1.0605571   1

By comparing the true edges and false edges similarity reported by Deepwalk and Node2Vec, we can see that Node2Vec is capable of embedding true edges with higher similarity, and false edges with lower similarity than Deepwalk, and therefore is considered as a better model.

# Link Prediction

Link prediction is a task to prediction unseen edges based on graph information. Let's use cross validation to check their performance in the link prediction task.

In [16]:
def case(kfold=5, node_dim=10, num_walks=10, walk_length=5, p=0.5, q=0.5):
    np.random.seed(0)

    deepwalk_auc_scores = list()
    node2vec_auc_scores = list()
    kf = KFold(n_splits=kfold, shuffle=True)
    for k, (train_idx, valid_idx) in enumerate(kf.split(edges)):
        # split edges into training and validation
        train_edges = [edges[idx] for idx in train_idx]
        valid_edges = [edges[idx] for idx in valid_idx]
        # generate the same validation size of false edges
        false_edges = generate_false_edges(edges, num_false_edges=len(valid_edges))

        # construct the graph and preprocess transition probabilities
        graph = construct_graph_from_edges(train_edges)
        alias_nodes, alias_edges = preprocess_transition_probs(graph, p=p, q=q)

        # build models and get auc scores
        model = build_deepwalk(graph, alias_nodes,
                               node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
        deepwalk_auc_scores.append(get_auc_score(model, valid_edges, false_edges))

        model = build_node2vec(graph, alias_nodes, alias_edges,
                               node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
        node2vec_auc_scores.append(get_auc_score(model, valid_edges, false_edges))

    deepwalk_auc_scores = np.array(deepwalk_auc_scores)
    node2vec_auc_scores = np.array(node2vec_auc_scores)
    print("DeepWalk: avg auc score: %.4f\tstd: %.4f" % (deepwalk_auc_scores.mean(), deepwalk_auc_scores.std()))
    print("node2vec: avg auc score: %.4f\tstd: %.4f" % (node2vec_auc_scores.mean(), node2vec_auc_scores.std()))

In [17]:
case(kfold=5, node_dim=2, num_walks=2, walk_length=2, p=0.5, q=0.5)

edges have mean weights 1.0 with std 0.0
number of nodes: 3574
number of edges: 12550
building a DeepWalk model...	number of walks: 7148	average walk length: 2.0000	build time: 1.0755
building a node2vec model...	number of walks: 7148	average walk length: 2.0000	build time: 1.0965
edges have mean weights 1.0 with std 0.0
number of nodes: 3610
number of edges: 12550
building a DeepWalk model...	number of walks: 7220	average walk length: 2.0000	build time: 1.1728
building a node2vec model...	number of walks: 7220	average walk length: 2.0000	build time: 1.0512
edges have mean weights 1.0 with std 0.0
number of nodes: 3639
number of edges: 12550
building a DeepWalk model...	number of walks: 7278	average walk length: 2.0000	build time: 0.9987
building a node2vec model...	number of walks: 7278	average walk length: 2.0000	build time: 1.0017
edges have mean weights 1.0 with std 0.0
number of nodes: 3582
number of edges: 12551
building a DeepWalk model...	number of walks: 7164	average walk leng

In [18]:
case(kfold=5, node_dim=5, num_walks=5, walk_length=5, p=0.5, q=0.5)

edges have mean weights 1.0 with std 0.0
number of nodes: 3574
number of edges: 12550
building a DeepWalk model...	number of walks: 17870	average walk length: 4.4799	build time: 2.9830
building a node2vec model...	number of walks: 17870	average walk length: 4.9491	build time: 6.9996
edges have mean weights 1.0 with std 0.0
number of nodes: 3610
number of edges: 12550
building a DeepWalk model...	number of walks: 18050	average walk length: 4.5289	build time: 2.9187
building a node2vec model...	number of walks: 18050	average walk length: 4.9625	build time: 6.7321
edges have mean weights 1.0 with std 0.0
number of nodes: 3639
number of edges: 12550
building a DeepWalk model...	number of walks: 18195	average walk length: 4.4843	build time: 2.9843
building a node2vec model...	number of walks: 18195	average walk length: 4.9532	build time: 7.2828
edges have mean weights 1.0 with std 0.0
number of nodes: 3582
number of edges: 12551
building a DeepWalk model...	number of walks: 17910	average wa

In [19]:
case(kfold=5, node_dim=10, num_walks=10, walk_length=10, p=0.5, q=0.5)

edges have mean weights 1.0 with std 0.0
number of nodes: 3574
number of edges: 12550
building a DeepWalk model...	number of walks: 35740	average walk length: 8.5227	build time: 7.4978
building a node2vec model...	number of walks: 35740	average walk length: 9.7371	build time: 15.8654
edges have mean weights 1.0 with std 0.0
number of nodes: 3610
number of edges: 12550
building a DeepWalk model...	number of walks: 36100	average walk length: 8.6765	build time: 7.5365
building a node2vec model...	number of walks: 36100	average walk length: 9.8225	build time: 42.9511
edges have mean weights 1.0 with std 0.0
number of nodes: 3639
number of edges: 12550
building a DeepWalk model...	number of walks: 36390	average walk length: 8.5785	build time: 10.0851
building a node2vec model...	number of walks: 36390	average walk length: 9.7888	build time: 42.4838
edges have mean weights 1.0 with std 0.0
number of nodes: 3582
number of edges: 12551
building a DeepWalk model...	number of walks: 35820	averag

With the help of p and q, the node2vec model can fit training data better. And you can have a try if you set p=q=1, the two models will return the same results.